<a href="https://colab.research.google.com/github/gianluigilopardo/Open-World-Recognition/blob/main/our_modifications/incremental_learning/ICARL/exemplars/iCaRLMain_Exemplars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [1]:
import os
import logging
import sys

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
if not os.path.isdir('./owr'):
  !git clone https://github.com/gianluigilopardo/Open-World-Recognition.git
  !mv 'Open-World-Recognition' 'owr'

Cloning into 'Open-World-Recognition'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 453 (delta 133), reused 146 (delta 107), pack-reused 257
Receiving objects: 100% (453/453), 2.06 MiB | 35.24 MiB/s, done.
Resolving deltas: 100% (276/276), done.


# Import packages

In [4]:
from owr.our_modifications.incremental_learning.ICARL.exemplars import ResNet
from owr.our_modifications.incremental_learning.ICARL.exemplars.dataset import Subset
from owr.our_modifications.incremental_learning.ICARL.exemplars.icarl import classify
from owr.our_modifications.incremental_learning.ICARL.exemplars.icarl import incremental_train
from owr.our_modifications.incremental_learning.ICARL.exemplars.icarl import update_representation
from owr.our_modifications.incremental_learning.ICARL.exemplars.icarl import construct_exemplar_set
from owr.our_modifications.incremental_learning.ICARL.exemplars.icarl import reduce_exemplars
from owr.our_modifications.incremental_learning.ICARL.exemplars.icarl import generate_new_exemplars
from owr.our_modifications.incremental_learning.ICARL.exemplars.models import compute_loss
from owr.our_modifications.incremental_learning.ICARL.exemplars.models import train_network
from owr.our_modifications.incremental_learning.ICARL.exemplars import params
from owr.our_modifications.incremental_learning.ICARL.exemplars import utils
from owr.our_modifications.incremental_learning.ICARL.exemplars.utils import get_classes_names
from owr.our_modifications.incremental_learning.ICARL.exemplars.utils import get_task_indexes
from owr.our_modifications.incremental_learning.ICARL.exemplars.utils import splitter
from owr.our_modifications.incremental_learning.ICARL.exemplars.utils import map_splits
from owr.our_modifications.incremental_learning.ICARL.exemplars.utils import get_classes
from owr.our_modifications.incremental_learning.ICARL.exemplars.utils import get_indexes
import owr.our_modifications.incremental_learning.ICARL.exemplars.models

In [5]:
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import random

In [6]:
print(params.SEED)
print(params.NUM_WORKERS)

42
2


In [7]:
print(params.NUM_EPOCHS)

70


# Define Datasets

In [8]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [9]:
from torchvision import datasets
trainDS = datasets.cifar.CIFAR100( 'data', train=True, download=True, transform=train_transformer)
testDS = datasets.cifar.CIFAR100( 'data', train=False, download=True, transform=test_transformer)


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [10]:
splits = splitter()

In [11]:
splits

[[91, 49, 74, 23, 27, 21, 24, 85, 87, 11],
 [55, 81, 28, 32, 25, 29, 95, 97, 12, 20],
 [62, 92, 34, 38, 31, 35, 93, 13, 26, 47],
 [70, 40, 44, 37, 41, 94, 14, 66, 69, 9],
 [76, 48, 86, 64, 16, 18, 15, 17, 96, 72],
 [88, 54, 75, 22, 33, 19, 30, 84, 5, 10],
 [63, 90, 43, 46, 42, 45, 83, 6, 39, 65],
 [77, 56, 82, 67, 7, 8, 80, 79, 78, 2],
 [60, 98, 50, 51, 89, 4, 3, 58, 99, 1],
 [53, 68, 71, 36, 61, 52, 0, 59, 57, 73]]

# Define Network

In [12]:
ICaRL = ResNet.resnet32(num_classes=100)
ICaRL =  ICaRL.to(params.DEVICE)

In [13]:
exemplars = [None]*params.NUM_CLASSES

test_indexes =  []
accs_train = []
accs_test = []

In [ ]:
for task in range(0, params.NUM_TASKS*params.TASK_SIZE, params.TASK_SIZE):
    train_indexes = get_task_indexes(trainDS, task)
    test_indexes = test_indexes + get_task_indexes(testDS, task)

    train_dataset = Subset(trainDS, train_indexes, transform = train_transformer)
    test_dataset = Subset(testDS, test_indexes, transform = test_transformer)

    train_loader = DataLoader( train_dataset, num_workers=2, batch_size=params.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader( test_dataset, num_workers=2, batch_size=params.BATCH_SIZE , shuffle=True )
    print(task)
    ICaRL, exemplars = incremental_train(trainDS, ICaRL, exemplars, task, train_transformer)
    col = []
    for i,x in enumerate( splits[ :int(task/10) + 1]) : 
        v = np.array(x)
        col = np.concatenate( (col,v), axis = None)
        col = col.astype(int)
    mean = None
    total = 0.0
    running_corrects = 0.0
    
    for img, lbl, _ in train_loader:
        img = img.float().to(params.DEVICE)
        preds, mean = classify(img, exemplars, ICaRL, task, trainDS, mean)
        preds = preds.to(params.DEVICE)
        labels = map_splits(lbl, col).to(params.DEVICE)
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'train accuracy = {accuracy}')
    accs_train.append(accuracy)

    total = 0.0
    running_corrects = 0.0
    tot_preds = []
    tot_lab = []
    for img, lbl, _ in test_loader:
        img = img.float().to(params.DEVICE)
        preds, _ = classify(img, exemplars, ICaRL, task, trainDS, mean)
        preds = preds.to(params.DEVICE)
        labels = map_splits(lbl, col).to(params.DEVICE)
        tot_preds = np.concatenate( ( tot_preds, preds.data.cpu().numpy() ) )
        tot_lab = np.concatenate( (tot_lab, labels.data.cpu().numpy()  ) )
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'test accuracy = {accuracy}')
    accs_test.append(accuracy)
    PATH = "./model" + str(task) + ".pr"
    torch.save(ICaRL, PATH)

0
Step: 0, Epoch: 0, Loss: 0.026603564620018005, Accuracy: 0.2082
Step: 0, Epoch: 1, Loss: 0.025619950145483017, Accuracy: 0.4176
Step: 0, Epoch: 2, Loss: 0.017441872507333755, Accuracy: 0.4788
Step: 0, Epoch: 3, Loss: 0.019961867481470108, Accuracy: 0.5292


In [ ]:
#Locura sugli exemplars